# FMA: A Dataset For Music Analysis

Michaël Defferrard, Kirell Benzi, Pierre Vandergheynst, Xavier Bresson, EPFL LTS2.

## Generation / Collection / Creation

From `raw_*.csv`, this notebook generates:
* `tracks.csv`: per-track / album / artist metadata.
* `genres.csv`: genre hierarchy.

A companion script, `creation.py`:
1. Download metadata through the API and store them in `raw_tracks.csv`, `raw_albums.csv`, `raw_artists.csv` and `raw_genres.csv`.
2. Download the full audio for each track.
3. Trim the audio to 30s clips.
4. Normalize the permissions and modification / acess times.
5. Create the `.zip` archives.

Todo
* update counts

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import ast
import pickle
import numpy as np
import pandas as pd
import IPython.display as ipd

In [ ]:
import utils
AUDIO_DIR = os.environ.get('AUDIO_DIR')
BASE_DIR = os.path.abspath(os.path.dirname(AUDIO_DIR))
FMA_FULL = os.path.join(BASE_DIR, 'fma_full')
FMA_LARGE = os.path.join(BASE_DIR, 'fma_large')
FMA_MEDIUM = os.path.join(BASE_DIR, 'fma_medium')
FMA_SMALL = os.path.join(BASE_DIR, 'fma_small')

## 1 Retrieve metadata and audio from FMA

1. Crawl the tracks, albums and artists metadata through their [API](https://freemusicarchive.org/api).
2. Download original `.mp3` by HTTPS for each track id (only if it does not exist already).

Todo:
* Scrap curators.
* Download images (`track_image_file`, `album_image_file`, `artist_image_file`). Beware the quality.
* Verify checksum for some random tracks.

Examples:
* To add new tracks: iterate from largest known track id to the most recent only.
* To update user data: get them all again.

In [ ]:
# Script used to query the API for all tracks, albums and artists.
# Then to download the audio through HTTPS.

# ./creation.py metadata
# ./creation.py data /path/to/fma/fma_full

#!cat creation.py

In [ ]:
# converters={'genres': ast.literal_eval}
tracks = pd.read_csv('raw_tracks.csv', index_col=0)
albums = pd.read_csv('raw_albums.csv', index_col=0)
artists = pd.read_csv('raw_artists.csv', index_col=0)
genres = pd.read_csv('raw_genres.csv', index_col=0)

not_found = pickle.load(open('not_found.pickle', 'rb'))

In [ ]:
audio_tids = []
for _, dirnames, files in os.walk(FMA_FULL):
    if dirnames == []:
        audio_tids.extend(int(file[:-4]) for file in files)

In [ ]:
tmp = tracks.shape[0], len(not_found['tracks'])
print('tracks: {} collected ({} not found)'.format(*tmp))
tmp = albums.shape[0], len(not_found['albums']), len(tracks['album_id'].unique())
print('albums: {} collected ({} not found, {} in tracks)'.format(*tmp))
tmp = artists.shape[0], len(not_found['artists']), len(tracks['artist_id'].unique())
print('artists: {} collected ({} not found, {} in tracks)'.format(*tmp))
print('genres: {} collected'.format(genres.shape[0]))
print('audio: {} collected ({} not found)'.format(len(audio_tids), len(not_found['audio'])))
assert len(set(tracks.index).intersection(audio_tids)) + len(not_found['audio']) == len(tracks.index)

In [ ]:
n = 5
#ipd.display(tracks.head(n))
#ipd.display(albums.head(n))
#ipd.display(artists.head(n))
#ipd.display(genres.head(n))

## 2 Format metadata

* Columns who are lists: genres, album_images, artist_images
* Fill `tracks.json` by iterating over all `track_id`.
* Fill `genres.json`
* Fill meta-data about encoding: length, number of samples, sample rate, bit rate, channels (mono/stereo), 16bits?.

Todo:
* Sanitize values, e.g. list of words for tags, valid links in `artist_wikipedia_page`.

In [ ]:
df, column = tracks, 'tags'
null = sum(df[column].isnull())
print('{} null, {} non-null'.format(null, df.shape[0] - null))
df[column].value_counts().head(10)

### 2.1 Tracks

In [ ]:
drop = [
    'license_image_file', 'license_image_file_large', 'license_parent_id', 'license_url',  # keep title only
    'track_file', 'track_image_file',  # used to download only
    'track_url', 'album_url', 'artist_url',  # only relevant on website
    'track_copyright_c', 'track_copyright_p',  # present for ~1000 tracks only
    # 'track_composer', 'track_lyricist', 'track_publisher',  # present for ~4000, <1000 and <2000 tracks
    'track_disc_number',  # different from 1 for <1000 tracks
    'track_explicit', 'track_explicit_notes',  # present for <4000 tracks
    'track_instrumental'  # ~6000 tracks have a 1, there is an instrumental genre
]
tracks.drop(drop, axis=1, inplace=True)
tracks.rename(columns={'license_title': 'track_license', 'tags': 'track_tags'}, inplace=True)

In [ ]:
def convert_duration(x):
    times = x.split(':')
    seconds = int(times[-1])
    minutes = int(times[-2])
    try:
        minutes += 60 * int(times[-3])
    except IndexError:
        pass
    return seconds + 60 * minutes

tracks['track_duration'] = tracks['track_duration'].map(convert_duration)

In [ ]:
def convert_datetime(df, column, format=None):
    df[column] = pd.to_datetime(df[column], infer_datetime_format=True, format=format)
convert_datetime(tracks, 'track_date_created')
convert_datetime(tracks, 'track_date_recorded')

In [ ]:
tracks['album_id'].fillna(-1, inplace=True)
tracks['track_bit_rate'].fillna(-1, inplace=True)
tracks = tracks.astype({'album_id': int, 'track_bit_rate': int})

In [ ]:
def convert_genres(genres):
    genres = ast.literal_eval(genres)
    return [int(genre['genre_id']) for genre in genres]

tracks['track_genres'].fillna('[]', inplace=True)
tracks['track_genres'] = tracks['track_genres'].map(convert_genres)

In [ ]:
tracks.columns

### 2.2 Albums

In [ ]:
drop = [
    'artist_name', 'album_url', 'artist_url',  # in tracks already (though it can be different)
    'album_handle',
    'album_image_file', 'album_images',  # todo: shall be downloaded
    #'album_producer', 'album_engineer',  # present for ~2400 albums only
]
albums.drop(drop, axis=1, inplace=True)
albums.rename(columns={'tags': 'album_tags'}, inplace=True)

In [ ]:
convert_datetime(albums, 'album_date_created')
convert_datetime(albums, 'album_date_released')

In [ ]:
albums.columns

### 2.3 Artists

In [ ]:
drop = [
    'artist_website', 'artist_url',  # in tracks already (though it can be different)
    'artist_handle',
    'artist_image_file', 'artist_images',  # todo: shall be downloaded
    'artist_donation_url', 'artist_paypal_name', 'artist_flattr_name',  # ~1600 & ~400 & ~70, not relevant
    'artist_contact',  # ~1500, not very useful data
    # 'artist_active_year_begin', 'artist_active_year_end',  # ~1400, ~500 only
    # 'artist_associated_labels',  # ~1000
    # 'artist_related_projects',  # only ~800, but can be combined with bio
]
artists.drop(drop, axis=1, inplace=True)
artists.rename(columns={'tags': 'artist_tags'}, inplace=True)

In [ ]:
convert_datetime(artists, 'artist_date_created')
for column in ['artist_active_year_begin', 'artist_active_year_end']:
    artists[column].replace(0.0, np.nan, inplace=True)
    convert_datetime(artists, column, format='%Y.0')

In [ ]:
artists.columns

### 2.4 Merge DataFrames

In [ ]:
not_found['albums'].remove(None)
not_found['albums'].append(-1)
not_found['albums'] = [int(i) for i in not_found['albums']]
not_found['artists'] = [int(i) for i in not_found['artists']]

In [ ]:
tracks = tracks.merge(albums, left_on='album_id', right_index=True, sort=False, how='left', suffixes=('', '_dup'))

n = sum(tracks['album_title_dup'].isnull())
print('{} tracks without extended album information ({} tracks without album_id)'.format(
    n, sum(tracks['album_id'] == -1)))
assert sum(tracks['album_id'].isin(not_found['albums'])) == n
assert sum(tracks['album_title'] != tracks['album_title_dup']) == n

tracks.drop('album_title_dup', axis=1, inplace=True)
assert not any('dup' in col for col in tracks.columns)

In [ ]:
# Album artist can be different than track artist. Keep track artist.
#tracks[tracks['artist_name'] != tracks['artist_name_dup']].select(lambda x: 'artist_name' in x, axis=1)

In [ ]:
tracks = tracks.merge(artists, left_on='artist_id', right_index=True, sort=False, how='left', suffixes=('', '_dup'))

n = sum(tracks['artist_name_dup'].isnull())
print('{} tracks without extended artist information'.format(n))
assert sum(tracks['artist_id'].isin(not_found['artists'])) == n
assert sum(tracks['artist_name'] != tracks[('artist_name_dup')]) == n

tracks.drop('artist_name_dup', axis=1, inplace=True)
assert not any('dup' in col for col in tracks.columns)

In [ ]:
columns = []
for name in tracks.columns:
    names = name.split('_')
    columns.append((names[0], '_'.join(names[1:])))
tracks.columns = pd.MultiIndex.from_tuples(columns)
assert all(label in ['track', 'album', 'artist'] for label in tracks.columns.get_level_values(0))

In [ ]:
# Todo: fill other columns ?
tracks['album', 'tags'].fillna('[]', inplace=True)
tracks['artist', 'tags'].fillna('[]', inplace=True)

columns = [('album', 'favorites'), ('album', 'comments'), ('album', 'listens'), ('album', 'tracks'),
           ('artist', 'favorites'), ('artist', 'comments')]
for column in columns:
    tracks[column].fillna(-1, inplace=True)
columns = {column: int for column in columns}
tracks = tracks.astype(columns)

## 3 Data cleaning

Todo
* Duplicates

In [ ]:
def lost(n):
    print('{} lost, {} left'.format(n - tracks.shape[0], tracks.shape[0]))
    return tracks.shape[0]

n = lost(tracks.shape[0])

In [ ]:
# Audio not found.
tracks.drop(not_found['audio'], inplace=True)
n = lost(n)

In [ ]:
# License forbids redistribution.
tracks = tracks[tracks[('track', 'license')] != 'FMA-Limited: Download Only']
n = lost(n)
print('{} licenses'.format(len(tracks[('track', 'license')].unique())))

In [ ]:
sum(tracks['track', 'title'].duplicated())
#sum(tracks['track', 'title'].isnull())

## 4 Genres

In [ ]:
genres.drop(['genre_handle', 'genre_color'], axis=1, inplace=True)
genres.rename(columns={'genre_parent_id': 'parent_id', 'genre_title': 'title'}, inplace=True)

In [ ]:
genres['parent_id'].fillna(0, inplace=True)
genres = genres.astype({'parent_id': int})

In [ ]:
# 13 (Easy Listening) has parent 126 which is missing
# --> a root genre on the website, although not in the genre menu
genres.at[13, 'parent_id'] = 0

# 580 (Abstract Hip-Hop) has parent 1172 which is missing
# --> listed as child of Hip-Hop on the website
genres.at[580, 'parent_id'] = 21

# 810 (Nu-Jazz) has parent 51 which is missing
# --> listed as child of Easy Listening on website
genres.at[810, 'parent_id'] = 13

# 763 (Holiday) has parent 763 which is itself
# --> listed as child of Sound Effects on website
genres.at[763, 'parent_id'] = 16

# Todo: should novely be under Experimental? It is alone on website.

In [ ]:
# Genre 806 (hiphop) should not exist. Replace it by 21 (Hip-Hop).
print('{} tracks have genre 806'.format(
    sum(tracks['track', 'genres'].map(lambda genres: 806 in genres))))
def change_genre(genres):
    return [genre if genre != 806 else 21 for genre in genres]
tracks['track', 'genres'] = tracks['track', 'genres'].map(change_genre)
genres.drop(806, inplace=True)

In [ ]:
# Number of tracks per genre.
genres['#tracks'] = 0
for row in tracks['track', 'genres'].iteritems():
    for genre in row[1]:
        genres.at[genre, '#tracks'] += 1

In [ ]:
# Cumulative number of tracks.
def get_cumulative(genre):
    cumulative = genres.at[genre, '#tracks']
    for genre in genres[genres['parent_id'] == genre].index:
        cumulative += get_cumulative(genre)
    return cumulative
genres['#tracks_cumulated'] = genres.index.map(get_cumulative)

In [ ]:
# Be sure all genres are in genres.csv.
tracks_genres = set()
for row in tracks['track', 'genres'].iteritems():
    tracks_genres.update(row[1])
print('{} genres in dataset'.format(len(tracks_genres)))
assert tracks_genres.issubset(genres.index)
genres.loc[set(genres.index).difference(tracks_genres)]

In [ ]:
# Top-level genre.
def get_parent(genre):
    parent = genres.at[genre, 'parent_id']
    if parent == 0:
        return genre
    else:
        return get_parent(parent)
genres['top_level'] = genres.index.map(get_parent)

In [ ]:
def get_top_genres(track_genres):
    return list(set(genres.at[genre, 'top_level'] for genre in track_genres))
tracks['track', 'top_genres'] = tracks['track', 'genres'].map(get_top_genres)

In [ ]:
genres.head(10)

## 5 Splits: train, validation, test

Take into account:
* Artists may only appear on one side.
* Stratification: all characteristics (sampling rates) should be distributed equally.

## 6 Subsets: large, medium, small

* Select the subsets.
* Clip all tracks. Ignore the ones shorter than 30 seconds (~2000).

* Quality:
    * Technical: bit rate
    * Missing metadata
    * User data

Todo:
* update duration from `ffmpeg`

In [ ]:
# ./creation.py clips /path/to/fma

In [ ]:
fma_full = tracks['track', 'duration'] > 30
#not_found['clips']

In [ ]:
medium = tracks['track', 'top_genres'].map(len) == 1

## 7 Store

* Fill the archives and compute their checksum.
    * Tool: zipfile
* Set permissions and creation/modification/access times.

Todo:
* Checksum for each individual file? Store output of sha1sum in another file.

Directory structure:
* `fma_metadata.zip`
    * `tracks.csv`
    * `genres.csv`
* `fma_features.zip`
    * `features.csv`
    * `echonest.csv`
* `fma_full.zip` (967 GiB, last collection by Kirell in April 2016 was 752 GiB)
* `fma_large.zip`
* `fma_medium.zip`
* `fma_small.zip` (30G full length --> 3.4GiB)

In [ ]:
for dataset in 'tracks', 'genres':
    eval(dataset).sort_index(axis=0, inplace=True)
    eval(dataset).sort_index(axis=1, inplace=True)
    eval(dataset).to_csv(dataset + '.csv')

In [ ]:
# Todo: ordered categories (license, subset, split, album type, artist bio?)
# a = tracks['license'].astype('category')
# Todo: class which inherit from DataFrame
# Todo: dtypes = {}
tracks = pd.read_csv('tracks.csv', index_col=0, header=[0, 1])

for column in [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'), ('track', 'genres')]:
    tracks[column] = tracks[column].map(ast.literal_eval)

dates = [('track', 'date_created'), ('track', 'date_recorded'),
         ('album', 'date_created'), ('album', 'date_released'),
         ('artist', 'date_created'), ('artist', 'active_year_begin'), ('artist', 'active_year_end')]
for column in dates:
    tracks[column] = pd.to_datetime(tracks[column])

tracks.dtypes

In [ ]:
README = """This .zip archive is part of the FMA, a dataset for music analysis.
Code & data: https://github.com/mdeff/fma
Paper: https://arxiv.org/abs/1612.01840

Each .mp3 is licensed by the corresponding artist listed in tracks.csv.

You can verify the integrity of the uncompressed files with sha1sum -c checksums.
It's sha1 checksum should be XXXXXXXXXXXXXX
"""
for dst in next(os.walk(BASE_DIR))[1]:
    dst = os.path.join(BASE_DIR, dst, 'README')
    try:
        os.chmod(dst, 0o666)
    except:
        pass
    with open(dst, 'w') as f:
        f.write(README)

In [ ]:
# ./creation.py normalize /path/to/fma
# ./creation.py zip

In [ ]:
# Create .zip archives.
# TODO: use zipfile
#shutil.make_archive(DST_DIR, 'zip', par_dir, ARCHIVE)
#os.utime(DST_DIR + '.zip', (TIME, TIME))
#os.chmod(DST_DIR + '.zip', 0o444)